In [1]:
import pandas as pd
import numpy as np

In [2]:
survey_raw = pd.read_excel('survey.xlsx', index_col=0) 
travel_raw = pd.read_excel('travel.xlsx') 

/Users/georgievakristina/.pyenv/versions/3.11.2/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
survey_raw.columns = ['name', 'lob', 'role', 'location', 'attending', '9 (Q9) ',
       '12 (Q12_1) ', '12 (Q12_2) ', '12 (Q12_3) ', '12 (Q12_4) ',
       '12 (Q12_5) ', '12 (Q12_6) ', '12 (Q12_7) ', '12 (Q12SPECIFIED_6) ',
       'travel_notes', 'doing-talk', 'talk-title', 'talk-description', '18 (Q18) ',
       'lob_rep_interest', '20 (Q20) ', '21 (Q21) ', '23 (Q23) ', 'lob_rep_reason',
       '25 (Q25_1) ', '26 (Q26_1) ', 'topics', 'other', 'Started',
       'Completed', 'Branched Out', 'Over Quota', 'Last Modified']

In [4]:
travel_raw["is_host"] = travel_raw['Location'] == travel_raw['Travel to']
travel = travel_raw[['Location', 'Travel to', 'People', 'Est total travel pp', 'is_host']].dropna().copy()
travel['People'] = travel['People'].astype(int)
travel.columns = ['location', 'destination', 'total_people', 'estimated_travel', 'is_host']
travel['location'] = travel['location'].str.lower()
travel['destination'] = travel['destination'].str.lower()
travel['location'] = travel['location'].str.replace('washington, d.c.', 'washington dc')
travel['destination'] = travel['destination'].str.replace('washington, d.c.', 'washington dc')
travel['destination'] = travel['destination'].str.replace('ã', 'a')
travel['location'] = travel['location'].str.replace('ã', 'a')

/var/folders/4w/v5gq_r_d3x5bfsq8nd08gj0c0000gn/T/ipykernel_68930/1157744701.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  travel['location'] = travel['location'].str.replace('washington, d.c.', 'washington dc')
/var/folders/4w/v5gq_r_d3x5bfsq8nd08gj0c0000gn/T/ipykernel_68930/1157744701.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  travel['destination'] = travel['destination'].str.replace('washington, d.c.', 'washington dc')


In [5]:
survey_raw = survey_raw[survey_raw['location'].str.len() >1]
survey_raw = survey_raw[survey_raw['attending'] == 'Yes']
survey_raw = survey_raw[survey_raw['location'] != 'dfasd']

In [6]:
#clean locations
survey_raw['location'] = survey_raw['location'].str.lower()
survey_raw['location'] = survey_raw['location'].str.replace('chk - ', '')
survey_raw['location'] = survey_raw['location'].str.replace('acc', '')
survey_raw['location'] = survey_raw['location'].str.replace('dal', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('guk', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('bos', 'boston')
survey_raw['location'] = survey_raw['location'].str.replace('bostonton', 'boston')
survey_raw['location'] = survey_raw['location'].str.replace('cas', 'casablanca')
survey_raw['location'] = survey_raw['location'].str.replace('casablancaablanca', 'casablanca')
survey_raw['location'] = survey_raw['location'].str.replace(',', '')
survey_raw['location'] = survey_raw['location'].str.replace('-', '')
survey_raw['location'] = survey_raw['location'].str.replace('dallaslaslas', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('dallaslas', 'dallas')
survey_raw['location'] = survey_raw['location'].str.replace('á', 'a')
survey_raw['location'] = survey_raw['location'].str.replace('ã', 'a')
survey_raw['location'] = survey_raw['location'].str.replace('ü', 'u')
survey_raw['location'] = survey_raw['location'].str.replace(' india', '')
survey_raw['location'] = survey_raw['location'].str.replace(' south africa', '')
survey_raw['location'] = survey_raw['location'].str.replace(' morocco', '')
survey_raw['location'] = survey_raw['location'].str.replace('gurgaon', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('gurugaon', 'gurugram')
survey_raw['location'] = survey_raw['location'].str.replace('bangalore', 'bengaluru')
survey_raw['location'] = survey_raw['location'].str.replace('  bek', '')
survey_raw['location'] = survey_raw['location'].str.replace('santaigo', 'santiago')
survey_raw['location'] = survey_raw['location'].str.replace(' brazil', '')
survey_raw['location'] = survey_raw['location'].str.replace(' city', '')
survey_raw['location'] = survey_raw['location'].str.replace('sfo', 'san francisco')
survey_raw['location'] = survey_raw['location'].str.replace('was', 'washington dc')
survey_raw['location'] = survey_raw['location'].str.replace('dchington dc', 'dc')
survey_raw['location'] = survey_raw['location'].str.replace('saettle', 'seattle')
survey_raw['location'] = survey_raw['location'].str.replace('colgone', 'cologne')
survey_raw['location'] = survey_raw['location'].str.replace('los angeles', 'temp')
survey_raw['location'] = survey_raw['location'].str.replace('manhattan beach', 'temp')
survey_raw['location'] = survey_raw['location'].str.replace('temp', 'manhattan beach & la')
survey_raw['location'] = survey_raw['location'].str.strip()

In [7]:
unique_locations = survey_raw['location'].unique()
unique_locations.sort()
unique_locations

array(['amsterdam', 'atlanta', 'austin', 'barcelona', 'bek', 'bengaluru',
       'berlin', 'bogota', 'boston', 'brooklyn', 'brussels', 'casablanca',
       'chicago', 'cologne', 'copenhagen', 'dallas', 'denver',
       'dusseldorf', 'frankfurt', 'gurugram', 'helsinki', 'houston',
       'india', 'johannesburg', 'london', 'madrid',
       'manhattan beach & la', 'melbourne', 'miami', 'milan', 'mumbai',
       'munich', 'new delhi', 'new jersey', 'new york', 'nyc', 'oslo',
       'paris', 'perth', 'pittsburgh', 'rio de janeiro', 'san francisco',
       'santiago', 'sao paulo', 'sdasd', 'seattle', 'seoul', 'shanghai',
       'shenzhen', 'silicon valley', 'singapore', 'stockholm', 'sydney',
       'tokyo', 'toronto', 'vienna', 'warsaw', 'washington dc', 'zurich'],
      dtype=object)

# General counts

In [8]:
survey_raw['doing-talk-clean'] =  survey_raw['doing-talk'] ==  'Yes'
survey_raw['doing-talk-clean'] = np.where(survey_raw['doing-talk-clean'] == False, np.nan, survey_raw['doing-talk-clean'])
#survey_raw[['doing-talk-clean', 'doing-talk'] ] 
office_counts = survey_raw.groupby('location').count()[['name', 'doing-talk-clean']]

In [9]:
combined = office_counts.join(travel.set_index('location'), how='left')
combined.columns = ['total_registered', 'total_doing_talk', 'destination', 'total_people', 'estimated_travel', 'is_host']

In [10]:
combined[combined['is_host'] == True].reset_index()

,location,total_registered,total_doing_talk,destination,total_people,estimated_travel,is_host
0,barcelona,7,0,barcelona,27.0,0,True
1,bengaluru,23,2,bengaluru,140.0,0,True
2,berlin,25,7,berlin,51.0,0,True
3,bogota,10,0,bogota,20.0,0,True
4,boston,34,3,boston,97.0,0,True
5,casablanca,13,6,casablanca,43.0,0,True
6,chicago,33,2,chicago,68.0,0,True
7,dallas,4,0,dallas,13.0,0,True
8,frankfurt,5,1,frankfurt,19.0,0,True
9,gurugram,35,5,gurugram,169.0,0,True


In [11]:
combined.to_csv("data/extracted_registrations.csv")

# Travellers list

In [12]:
travel_subset = travel[travel['total_people'] > 1]
merged_with_travel = pd.merge(survey_raw,travel_subset,how='left',left_on='location',right_on='location')
small_office_travellers = merged_with_travel[merged_with_travel['is_host'] == False].copy()

In [13]:
small_office_travellers['estimated_travel_plus_perc'] = small_office_travellers['estimated_travel'] * 1.30
small_office_travellers[['name', 'role', 'lob', 'location', 'destination', 'estimated_travel_plus_perc', 'doing-talk-clean', 'travel_notes', 'other']]

,name,role,lob,location,destination,estimated_travel_plus_perc,doing-talk-clean,travel_notes,other
0,Eivind Strøm,Data Scientist,X AI & Software,oslo,stockholm,364.0,NaN,NaN,.
1,Valentina Jarovaja,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,NaN
3,Marion Nöske,Senior Director Data & Analytics - X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,thank you for organizing!
5,Michael Logen,Data & Analytics Lead,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,-
19,Martin Schleicher,Data & Analytics Lead – X Delivery,X Delivery,dusseldorf,frankfurt,546.0,NaN,NaN,Don't make optional questions mandatory.
...,...,...,...,...,...,...,...,...,...
547,Tom Sandeman,Principal,X AI & Software,melbourne,sydney,455.0,1.0,Potentially client requirements,NaN
550,Brittany Woods,Sr. Solution Analyst,X Delivery,manhattan beach & la,manhattan beach,0.0,NaN,NaN,no
552,Heidi He,Data Scientist,X AI & Software,melbourne,sydney,455.0,NaN,NaN,NaN
570,Michael Boselowitz,Principle Software Design Engineer,X AI & Software,pittsburgh,washington dc,911.3,NaN,No,NaN


# Host office representation

In [14]:
lob_counts = merged_with_travel.groupby(['destination', 'lob']).count()['name'].reset_index()
lob_counts_pivoted = pd.pivot(lob_counts, columns=['lob'], index=['destination'])['name'][['X AI & Software', 'X Delivery', 'X Ventures']]

In [15]:
lob_counts_pivoted

lob,X AI & Software,X Delivery,X Ventures
destination,,,
barcelona,3.0,4.0,NaN
bengaluru,3.0,22.0,NaN
berlin,11.0,2.0,20.0
bogota,1.0,9.0,NaN
boston,17.0,17.0,NaN
casablanca,1.0,11.0,1.0
chicago,10.0,22.0,1.0
dallas,14.0,1.0,1.0
frankfurt,2.0,11.0,NaN


In [16]:
lob_rep_plan = pd.read_excel('lob_rep_plan.xlsx') 
lob_rep_plan['Missing LOB'] = lob_rep_plan['Missing LOB'].str.split(',')
lob_rep_plan = lob_rep_plan.explode(['Missing LOB'])

# LOB reps

In [17]:
lob_reps = survey_raw[(~survey_raw['lob_rep_interest'].isna()) & (survey_raw['lob_rep_interest'] != 'None of the above')]


In [18]:
lob_reps.to_csv("data/extracted_lob_reps.csv")

### AI & software

In [19]:
lob_counts_pivoted[lob_counts_pivoted['X AI & Software'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,


In [20]:
lob_reps[lob_reps['lob'] == 'X AI & Software'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),...,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,
638295193090591825,Guillem Borrell Nogueras,X AI & Software,Lead AI Engineer,madrid,Yes,No,-,-,-,-,...,NaN,NaN,Anything that is interesting and not about GenAI,"Please, don't make it only about GenAI",2023-09-05 14:01:49,2023-09-05 14:13:08,NaN,NaN,2023-09-05 14:13:08,1.0
638298051723847524,Archit Matta,X AI & Software,Data Scientist,new york,Yes,No,-,-,-,-,...,NaN,NaN,I'd be interested in knowing about more networ...,I would also be super interested in helping pl...,2023-09-08 21:26:12,2023-09-08 21:34:28,NaN,NaN,2023-09-08 21:34:28,NaN
638301162078151552,Abby Gong,X AI & Software,Senior Data Scientist,boston,Yes,No,-,-,-,-,...,NaN,NaN,"Future one BCGx offering, collaborative ways o...",would like to hear the envision from org team ...,2023-09-12 11:50:07,2023-09-12 12:01:12,NaN,NaN,2023-09-12 19:37:38,1.0
638301202864066433,Kamal Darchini,X AI & Software,Lead data scientist,toronto,Yes,No,-,-,-,-,...,NaN,NaN,Hands on GenAI,Thanks for organizing!,2023-09-12 12:58:06,2023-09-12 13:11:28,NaN,NaN,2023-09-12 13:11:28,NaN
638301255487576553,Saurabh Mahajan,X AI & Software,AI Engineer,denver,Yes,Yes,-,Vegetarian,-,-,...,NaN,NaN,-,Can you send a broad categories and informatio...,2023-09-12 14:25:48,2023-09-12 14:40:44,NaN,NaN,2023-09-12 14:40:44,NaN
638312276035642564,Olga Kotova,X AI & Software,Data Scientist,london,Yes,No,-,-,-,-,...,NaN,NaN,Generative algorithms and language models,-,2023-09-25 08:33:23,2023-09-25 08:45:57,NaN,NaN,2023-09-25 08:45:57,1.0
638315193987323543,Kaoru Schwarzenegger,X AI & Software,Data Scientist,tokyo,Yes,No,-,-,-,-,...,NaN,NaN,Different perspectives from other LOBs and ide...,Will likely not be able to block the whole day...,2023-09-28 17:36:38,2023-09-28 17:43:25,NaN,NaN,2023-09-28 17:43:25,NaN
638315202876045630,Adam Benali,X AI & Software,AI Engineer,stockholm,Yes,No,-,-,-,-,...,NaN,NaN,-,-,2023-09-28 17:51:27,2023-09-28 17:57:33,NaN,NaN,2023-09-28 17:57:33,NaN
638315430336436294,Dave Rattan,X AI & Software,AI Engineer,sydney,Yes,No,-,-,-,-,...,NaN,NaN,Ops and infrastructure as code implementation,N/a,2023-09-29 00:10:33,2023-09-29 00:26:11,NaN,NaN,2023-09-29 00:26:11,1.0


### X Delivery

In [21]:
lob_counts_pivoted[lob_counts_pivoted['X Delivery'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,
milan,15.0,NaN,NaN
paris,33.0,NaN,NaN
santiago,11.0,NaN,NaN
singapore,14.0,NaN,8.0
sydney,27.0,NaN,3.0
tokyo,7.0,NaN,3.0
washington dc,9.0,NaN,NaN
zurich,7.0,NaN,NaN


In [22]:
lob_reps[lob_reps['lob'] == 'X Delivery'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),...,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,
638298890718873054,Amine Benaddi,X Delivery,Engineering Lead,casablanca,Yes,No,-,-,-,-,...,NaN,NaN,Architectures leveraged in other cases,"Comparing Astro, Nextjs, fresh, blitzjs, etc",2023-09-09 20:44:31,2023-09-09 20:49:39,NaN,NaN,2023-09-09 20:49:39,1.0
638307984685797820,Panagiota Papakosta,X Delivery,Senior Analyst,munich,Yes,No,-,-,-,-,...,NaN,NaN,other C&S applications,thanks for orga,2023-09-20 09:21:08,2023-09-20 09:27:17,NaN,NaN,2023-09-20 09:27:17,1.0
638309142512518582,Purnima Aggarwal,X Delivery,Lead data analyst,new delhi,Yes,Yes,Vegan,Vegetarian,-,-,...,NaN,NaN,Outlook,No,2023-09-21 17:30:51,2023-09-21 17:33:52,NaN,NaN,2023-09-21 17:34:01,NaN
638309193066578398,Amar Patel,X Delivery,Product Management Director,austin,Yes,Yes,-,-,-,-,...,NaN,NaN,GenAI use cases that we have developed and dep...,no,2023-09-21 18:55:06,2023-09-21 19:00:59,NaN,NaN,2023-09-21 19:00:59,NaN
638309418719094351,Jason He,X Delivery,Senior Delivery Analyst,shenzhen,Yes,Yes,-,-,-,-,...,NaN,NaN,Sharing (case / tool) from Venture team,No,2023-09-22 01:11:11,2023-09-22 01:20:59,NaN,NaN,2023-09-22 01:20:59,1.0
638312379032342134,Oliver Hof,X Delivery,Technical Delivery Sr. Manager,warsaw,Yes,Yes,-,-,-,-,...,NaN,NaN,How teams are using AI tools internally. Any r...,"I am not quite sure how the LOB Rep will work,...",2023-09-25 11:25:03,2023-09-25 11:30:19,NaN,NaN,2023-09-25 11:30:19,NaN
638314001137590077,mohd sharique ansari,X Delivery,engineering lead,gurugram,Yes,No,Vegan,Vegetarian,-,-,...,NaN,NaN,"Software Security, Infra modernization techniques",no,2023-09-27 08:28:33,2023-09-27 08:37:48,NaN,NaN,2023-09-27 08:37:48,NaN
638314079246651573,Madhur Mukhi,X Delivery,Engineering Lead,gurugram,Yes,No,-,Vegetarian,-,-,...,NaN,NaN,"Agile , Software development",NaN,2023-09-27 10:38:44,2023-09-27 10:52:54,NaN,NaN,2023-09-27 10:52:54,1.0
638315013930037917,Luísa Lichfett,X Delivery,Software Engineer,sao paulo,Yes,Yes,-,-,-,-,...,NaN,NaN,"GenAI, zero carbon technologies and approaches...",I think it's important for a community like BC...,2023-09-28 12:36:33,2023-09-28 12:47:15,NaN,NaN,2023-09-28 12:47:15,NaN


# Ventures

In [23]:
lob_counts_pivoted[lob_counts_pivoted['X Ventures'].isna()]

lob,X AI & Software,X Delivery,X Ventures
destination,,,
barcelona,3.0,4.0,NaN
bengaluru,3.0,22.0,NaN
bogota,1.0,9.0,NaN
boston,17.0,17.0,NaN
frankfurt,2.0,11.0,NaN
gurugram,2.0,37.0,NaN
madrid,9.0,14.0,NaN
milan,15.0,NaN,NaN
munich,7.0,4.0,NaN


In [24]:
lob_reps[lob_reps['lob'] == 'X Ventures'].dropna(subset=['lob_rep_reason'])

,name,lob,role,location,attending,9 (Q9),12 (Q12_1),12 (Q12_2),12 (Q12_3),12 (Q12_4),...,25 (Q25_1),26 (Q26_1),topics,other,Started,Completed,Branched Out,Over Quota,Last Modified,doing-talk-clean
Record ID,,,,,,,,,,,,,,,,,,,,,
638301182426033496,Robin Weston,X Ventures,Principal,singapore,Yes,No,-,-,-,-,...,NaN,NaN,Would love to hear about some ventures - I hav...,NaN,2023-09-12 12:24:02,2023-09-12 12:28:14,NaN,NaN,2023-09-12 12:28:14,1.0
638315196919892165,Sebastian Geidies,X Ventures,"Partner & Vice President, Engineering",berlin,Yes,No,-,-,-,-,...,NaN,NaN,Distributed Computing,NaN,2023-09-28 17:41:31,2023-09-28 17:43:46,NaN,NaN,2023-09-28 17:43:46,NaN
638315201269934155,Astera Schneeweisz,X Ventures,VP of Engineering,berlin,Yes,No,-,-,-,-,...,NaN,NaN,"Diversity, cultural topics, eng starter projects",not at this time :),2023-09-28 17:48:46,2023-09-28 17:56:42,NaN,NaN,2023-09-28 17:56:42,NaN
638315258771559218,Constantine Karlis,X Ventures,Lead Engineer,berlin,Yes,No,-,-,-,-,...,NaN,NaN,"In general: cats, noodles, synthesisers. Speci...",For the talk I would like to collaborate with ...,2023-09-28 19:24:37,2023-09-28 19:44:53,NaN,NaN,2023-09-28 19:44:53,1.0
